# PP2A - Single gene analysis in GP2 data

## Description

Using "individual level data"

### 0. Getting Started

- Loading Python libraries
- Defining functions
- Installing packages

### 1. Copy data from workspace to cloud environment

### 2. Extract PP2A

### 3. Annotate PP2A variants

### 4. Extract coding/non-syn variants

### 5. Calculate frequency in cases versus controls

### 6. Calculate frequency (homozygotes) in cases versus controls

### 7. Save out results

## Getting Started

### Loading Python libraries

### Defining functions

In [1]:
# Use the os package to interact with the environment
import os

# Bring in Pandas for Dataframe functionality
import pandas as pd

# Numpy for basics
import numpy as np

# Use StringIO for working with file contents
from io import StringIO

# Enable IPython to display matplotlib graphs
import matplotlib.pyplot as plt
%matplotlib inline

# Enable interaction with the FireCloud API
from firecloud import api as fapi

# Import the iPython HTML rendering for displaying links to Google Cloud Console
from IPython.core.display import display, HTML

# Import urllib modules for building URLs to Google Cloud Console
import urllib.parse

# BigQuery for querying data
from google.cloud import bigquery

#Import Sys
import sys as sys

In [2]:
# Utility routine for printing a shell command before executing it
def shell_do(command):
    print(f'Executing: {command}', file=sys.stderr)
    !$command
    
def shell_return(command):
    print(f'Executing: {command}', file=sys.stderr)
    output = !$command
    return '\n'.join(output)

# Utility routine for printing a query before executing it
def bq_query(query):
    print(f'Executing: {query}', file=sys.stderr)
    return pd.read_gbq(query, project_id=BILLING_PROJECT_ID, dialect='standard')

# Utility routine for display a message and a link
def display_html_link(description, link_text, url):
    html = f'''
    <p>
    </p>
    <p>
    {description}
    <a target=_blank href="{url}">{link_text}</a>.
    </p>
    '''

    display(HTML(html))

# Utility routines for reading files from Google Cloud Storage
def gcs_read_file(path):
    """Return the contents of a file in GCS"""
    contents = !gsutil -u {BILLING_PROJECT_ID} cat {path}
    return '\n'.join(contents)
    
def gcs_read_csv(path, sep=None):
    """Return a DataFrame from the contents of a delimited file in GCS"""
    return pd.read_csv(StringIO(gcs_read_file(path)), sep=sep, engine='python')

# Utility routine for displaying a message and link to Cloud Console
def link_to_cloud_console_gcs(description, link_text, gcs_path):
    url = '{}?{}'.format(
        os.path.join('https://console.cloud.google.com/storage/browser',
                     gcs_path.replace("gs://","")),
        urllib.parse.urlencode({'userProject': BILLING_PROJECT_ID}))

    display_html_link(description, link_text, url)

### Set paths

In [3]:
# Set up billing project and data path variables
BILLING_PROJECT_ID = os.environ['GOOGLE_PROJECT']
WORKSPACE_NAMESPACE = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE_NAME = os.environ['WORKSPACE_NAME']
WORKSPACE_BUCKET = os.environ['WORKSPACE_BUCKET']

WORKSPACE_ATTRIBUTES = fapi.get_workspace(WORKSPACE_NAMESPACE, WORKSPACE_NAME).json().get('workspace',{}).get('attributes',{})

## Print the information to check we are in the proper release and billing 
## This will be different for you, the user, depending on the billing project your workspace is on
print('Billing and Workspace')
print(f'Workspace Name: {WORKSPACE_NAME}')
print(f'Billing Project: {BILLING_PROJECT_ID}')
print(f'Workspace Bucket, where you can upload and download data: {WORKSPACE_BUCKET}')
print('')


## GP2 v3.0
## Explicitly define release v3.0 path 
GP2_RELEASE_PATH = 'gs://gp2tier2/release3_31102022'
GP2_CLINICAL_RELEASE_PATH = f'{GP2_RELEASE_PATH}/clinical_data'
GP2_RAW_GENO_PATH = f'{GP2_RELEASE_PATH}/raw_genotypes'
GP2_IMPUTED_GENO_PATH = f'{GP2_RELEASE_PATH}/imputed_genotypes'
print('GP2 v3.0')
print(f'Path to GP2 v3.0 Clinical Data: {GP2_CLINICAL_RELEASE_PATH}')
print(f'Path to GP2 v3.0 Raw Genotype Data: {GP2_RAW_GENO_PATH}')
print(f'Path to GP2 v3.0 Imputed Genotype Data: {GP2_IMPUTED_GENO_PATH}')


## AMP-PD v3.0
## Explicitly define release v3.0 path 
AMP_RELEASE_PATH = 'gs://amp-pd-data/releases/2022_v3release_1115'
AMP_CLINICAL_RELEASE_PATH = f'{AMP_RELEASE_PATH}/clinical'

AMP_WGS_RELEASE_PATH = 'gs://amp-pd-genomics/releases/2022_v3release_1115/wgs-WB-DWGS'
AMP_WGS_RELEASE_PLINK_PATH = os.path.join(AMP_WGS_RELEASE_PATH, 'plink')
AMP_WGS_RELEASE_PLINK_PFILES = os.path.join(AMP_WGS_RELEASE_PLINK_PATH, 'pfiles')

print('AMP-PD v3.0')
print(f'Path to AMP-PD v3.0 Clinical Data: {AMP_CLINICAL_RELEASE_PATH}')
print(f'Path to AMP-PD v3.0 WGS Data: {AMP_WGS_RELEASE_PLINK_PATH}')
print(f'Path to AMP-PD v3.0 WGS Data: {AMP_WGS_RELEASE_PLINK_PFILES}')
print('')

Billing and Workspace
Workspace Name: PP2A
Billing Project: terra-02e7cd23
Workspace Bucket, where you can upload and download data: gs://fc-a2382a6f-59cf-4cf3-b70e-8cd83806d5e0

GP2 v3.0
Path to GP2 v3.0 Clinical Data: gs://gp2tier2/release3_31102022/clinical_data
Path to GP2 v3.0 Raw Genotype Data: gs://gp2tier2/release3_31102022/raw_genotypes
Path to GP2 v3.0 Imputed Genotype Data: gs://gp2tier2/release3_31102022/imputed_genotypes
AMP-PD v3.0
Path to AMP-PD v3.0 Clinical Data: gs://amp-pd-data/releases/2022_v3release_1115/clinical
Path to AMP-PD v3.0 WGS Data: gs://amp-pd-genomics/releases/2022_v3release_1115/wgs-WB-DWGS/plink
Path to AMP-PD v3.0 WGS Data: gs://amp-pd-genomics/releases/2022_v3release_1115/wgs-WB-DWGS/plink/pfiles



### Install packages

#### Install Plink 1.9 and Plink 2.0

In [4]:
%%bash

mkdir -p ~/tools
cd ~/tools

if test -e /home/jupyter/tools/plink; then
echo "Plink1.9 is already installed in /home/jupyter/tools/"

else
echo -e "Downloading plink \n    -------"
wget -N http://s3.amazonaws.com/plink1-assets/plink_linux_x86_64_20190304.zip 
unzip -o plink_linux_x86_64_20190304.zip
echo -e "\n plink downloaded and unzipped in /home/jupyter/tools \n "

fi


if test -e /home/jupyter/tools/plink2; then
echo "Plink2 is already installed in /home/jupyter/tools/"

else
echo -e "Downloading plink2 \n    -------"
wget -N https://s3.amazonaws.com/plink2-assets/alpha3/plink2_linux_avx2_20220603.zip
unzip -o plink2_linux_avx2_20220603.zip
echo -e "\n plink2 downloaded and unzipped in /home/jupyter/tools \n "

fi

Plink1.9 is already installed in /home/jupyter/tools/
Plink2 is already installed in /home/jupyter/tools/


In [5]:
%%bash
ls /home/jupyter/tools/

annovar
annovar.latest.tar.gz
LICENSE
plink
plink2
plink2_linux_avx2_20220603.zip
plink_linux_x86_64_20190304.zip
prettify
toy.map
toy.ped


#### Remote restrictions

In [6]:
%%bash

# chmod plink 1.9 
chmod u+x /home/jupyter/tools/plink

In [7]:
%%bash

# chmod plink 2.0
chmod u+x /home/jupyter/tools/plink2

### Install ANNOVAR

In [8]:
%%bash

# Install ANNOVAR:
# https://www.openbioinformatics.org/annovar/annovar_download_form.php

if test -e /home/jupyter/tools/annovar; then

echo "annovar is already installed in /home/jupyter/tools/"
else
echo "annovar is not installed"
cd /home/jupyter/tools/

wget http://www.openbioinformatics.org/annovar/download/0wgxR2rIVP/annovar.latest.tar.gz

tar xvfz annovar.latest.tar.gz

fi

annovar is already installed in /home/jupyter/tools/


In [9]:
%%bash
ls /home/jupyter/tools/

annovar
annovar.latest.tar.gz
LICENSE
plink
plink2
plink2_linux_avx2_20220603.zip
plink_linux_x86_64_20190304.zip
prettify
toy.map
toy.ped


#### Install ANNOVAR: Download sources of annotation 

In [10]:
%%bash

cd /home/jupyter/tools/annovar/

perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar refGene humandb/
perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar clinvar_20140902 humandb/
#perl annotate_variation.pl -buildver hg38 -downdb cytoBand humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar ensGene humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar exac03 humandb/ 
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar avsnp147 humandb/ 
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar dbnsfp30a humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar gnomad211_genome humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar ljb26_all humandb/

NOTICE: Web-based checking to see whether ANNOVAR new version is available ... Done
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_refGene.txt.gz ... OK
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_refGeneMrna.fa.gz ... OK
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_refGeneVersion.txt.gz ... OK
NOTICE: Uncompressing downloaded files
NOTICE: Finished downloading annotation files for hg38 build version, with files saved at the 'humandb' directory
NOTICE: Web-based checking to see whether ANNOVAR new version is available ... Done
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_clinvar_20140902.txt.gz ... OK
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_clinvar_20140902.txt.idx.gz ... OK
NOTICE: Uncompressing downloaded files
NOTICE: Finished d

In [11]:
%%bash
ls /home/jupyter/tools/annovar/

annotate_variation.pl
coding_change.pl
convert2annovar.pl
example
humandb
retrieve_seq_from_fasta.pl
table_annovar.pl
variants_reduction.pl


## EASICAN

## Copy data from GP2 bucket to workspace

In [12]:
# Make a directory
print("Making a working directory")
WORK_DIR = f'/home/jupyter/PP2A_GP2/'
shell_do(f'mkdir -p {WORK_DIR}') # f' significa f-string - contiene expresiones para ejecutar el codigo

Making a working directory


Executing: mkdir -p /home/jupyter/PP2A_GP2/


In [13]:
# Check directory where AMP-PD data is
shell_do(f'gsutil -u {BILLING_PROJECT_ID} ls {GP2_IMPUTED_GENO_PATH}')

Executing: gsutil -u terra-02e7cd23 ls gs://gp2tier2/release3_31102022/imputed_genotypes


gs://gp2tier2/release3_31102022/imputed_genotypes/AAC/
gs://gp2tier2/release3_31102022/imputed_genotypes/AFR/
gs://gp2tier2/release3_31102022/imputed_genotypes/AJ/
gs://gp2tier2/release3_31102022/imputed_genotypes/AMR/
gs://gp2tier2/release3_31102022/imputed_genotypes/CAS/
gs://gp2tier2/release3_31102022/imputed_genotypes/EAS/
gs://gp2tier2/release3_31102022/imputed_genotypes/EUR/
gs://gp2tier2/release3_31102022/imputed_genotypes/MDE/
gs://gp2tier2/release3_31102022/imputed_genotypes/SAS/


In [14]:
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp -r {GP2_IMPUTED_GENO_PATH}/EAS/chr9* {WORK_DIR}')

Executing: gsutil -u terra-02e7cd23 -m cp -r gs://gp2tier2/release3_31102022/imputed_genotypes/EAS/chr9* /home/jupyter/PP2A_GP2/


Copying gs://gp2tier2/release3_31102022/imputed_genotypes/EAS/chr9_EAS_release3.pgen...
Copying gs://gp2tier2/release3_31102022/imputed_genotypes/EAS/chr9_EAS_release3.psam...
Copying gs://gp2tier2/release3_31102022/imputed_genotypes/EAS/chr9_EAS_release3.log...
Copying gs://gp2tier2/release3_31102022/imputed_genotypes/EAS/chr9_EAS_release3.pvar...
\ [4/4 files][125.5 MiB/125.5 MiB] 100% Done                                    
Operation completed over 4 objects/125.5 MiB.                                    


### Create a covariate file with GP2 data

In [15]:
shell_do(f'gsutil -u {BILLING_PROJECT_ID} ls {GP2_CLINICAL_RELEASE_PATH}')
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp -r {GP2_CLINICAL_RELEASE_PATH}/master_key_release3_final.csv {WORK_DIR}')

Executing: gsutil -u terra-02e7cd23 ls gs://gp2tier2/release3_31102022/clinical_data


gs://gp2tier2/release3_31102022/clinical_data/
gs://gp2tier2/release3_31102022/clinical_data/master_key_release3_final.csv
gs://gp2tier2/release3_31102022/clinical_data/release3_31102022_data_dictionary.csv


Executing: gsutil -u terra-02e7cd23 -m cp -r gs://gp2tier2/release3_31102022/clinical_data/master_key_release3_final.csv /home/jupyter/PP2A_GP2/


Copying gs://gp2tier2/release3_31102022/clinical_data/master_key_release3_final.csv...
/ [1/1 files][  1.6 MiB/  1.6 MiB] 100% Done                                    
Operation completed over 1 objects/1.6 MiB.                                      


In [16]:
cov = pd.read_csv(f'{WORK_DIR}/master_key_release3_final.csv')
cov.columns

Index(['GP2ID', 'GP2sampleID', 'manifest_id', 'phenotype', 'pheno_for_qc',
       'other_pheno', 'sex_for_qc', 'age', 'age_of_onset', 'age_at_diagnosis',
       'age_at_death', 'race_for_qc', 'family_history_for_qc', 'region_for_qc',
       'study', 'pruned', 'pruned_reason', 'label', 'related'],
      dtype='object')

In [17]:
cov_reduced = cov[['GP2sampleID','GP2sampleID','sex_for_qc', 'age', 'phenotype']]
cov_reduced.columns = ['FID','IID', 'SEX','AGE','PHENO']
cov_reduced

,FID,IID,SEX,AGE,PHENO
0,SYNAPS-KZ_000001_s1,SYNAPS-KZ_000001_s1,2,64.0,Control
1,SYNAPS-KZ_000002_s1,SYNAPS-KZ_000002_s1,1,53.0,Control
2,SYNAPS-KZ_000003_s1,SYNAPS-KZ_000003_s1,2,59.0,Control
3,SYNAPS-KZ_000004_s1,SYNAPS-KZ_000004_s1,1,57.0,Control
4,SYNAPS-KZ_000005_s1,SYNAPS-KZ_000005_s1,1,59.0,Control
...,...,...,...,...,...
16852,PPMI_000789_s1,PPMI_000789_s1,2,NaN,Other
16853,PPMI_000781_s1,PPMI_000781_s1,1,57.0,Other
16854,PPMI_000818_s1,PPMI_000818_s1,1,60.0,Other
16855,PPMI_000841_s1,PPMI_000841_s1,2,68.0,Other


In [18]:
conditions = [
    (cov_reduced['PHENO'] == "Case"),
    (cov_reduced['PHENO'] == "Control")]

In [19]:
choices = [2,1]
cov_reduced['PHENOTYPE'] = np.select(conditions, choices, default=-9).astype(np.int64)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [20]:
cov_reduced.reset_index(inplace=True)
cov_reduced.drop(columns=["index"], inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [21]:
cov_reduced.drop(columns=['PHENO'], inplace=True)

In [22]:
sex = cov_reduced[['FID','IID','SEX']]
sex.to_csv(f'{WORK_DIR}/SEX.txt',index=False, sep="\t")

In [23]:
pheno = cov_reduced[['FID','IID','PHENOTYPE']]
pheno.to_csv(f'{WORK_DIR}/PHENO.txt',index=False, sep="\t")

## Extract PP2A

In [24]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

# PODXL: gene positions on hg38 (from https://useast.ensembl.org/index.html)
/home/jupyter/tools/plink2 \
--pfile chr9_EAS_release3 \
--chr 9 \
--from-bp 129110950  \
--to-bp 129148946 \
--make-bed \
--out pheno_PP2A_EAS

PLINK v2.00a3.3LM AVX2 Intel (3 Jun 2022)      www.cog-genomics.org/plink/2.0/
(C) 2005-2022 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to pheno_PP2A_EAS.log.
Options in effect:
  --chr 9
  --from-bp 129110950
  --make-bed
  --out pheno_PP2A_EAS
  --pfile chr9_EAS_release3
  --to-bp 129148946

Start time: Fri Jun  2 08:10:27 2023
14998 MiB RAM detected; reserving 7499 MiB for main workspace.
Using up to 4 compute threads.
137 samples (52 females, 85 males; 137 founders) loaded from
chr9_EAS_release3.psam.
447232 variants loaded from chr9_EAS_release3.pvar.
1 binary phenotype loaded (104 cases, 33 controls).
150 variants remaining after main filters.
Writing pheno_PP2A_EAS.fam ... done.
Writing pheno_PP2A_EAS.bim ... done.
Writing pheno_PP2A_EAS.bed ... 0%done.
End time: Fri Jun  2 08:10:27 2023


### Visualize plink files bim, fam and bed

In [25]:
%%bash

# Visualize bim file
WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

head pheno_PP2A_EAS.bim

9	chr9:129110977:A:C	0	129110977	C	A
9	chr9:129111323:C:G	0	129111323	G	C
9	chr9:129111962:C:T	0	129111962	T	C
9	chr9:129112273:A:T	0	129112273	T	A
9	chr9:129112362:A:G	0	129112362	G	A
9	chr9:129112376:G:A	0	129112376	A	G
9	chr9:129113341:C:T	0	129113341	T	C
9	chr9:129113612:A:G	0	129113612	G	A
9	chr9:129113698:A:T	0	129113698	T	A
9	chr9:129113827:A:G	0	129113827	G	A


In [26]:
%%bash

# Visualize fam file
WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

head pheno_PP2A_EAS.fam

0	APGS_000051_s1	0	0	1	2
0	APGS_000113_s1	0	0	1	2
0	APGS_000135_s1	0	0	1	2
0	APGS_000186_s1	0	0	1	2
0	APGS_000201_s1	0	0	1	2
0	APGS_000204_s1	0	0	2	2
0	APGS_000263_s1	0	0	1	2
0	APGS_000314_s1	0	0	2	2
0	APGS_000415_s1	0	0	2	2
0	APGS_000422_s1	0	0	2	2


In [27]:
%%bash

# Visualize fam file
WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

head pheno_PP2A_EAS.bed

l�⧲�����{*���ϫ�������������������������������������������������  �   �     �  ��  ��  "  (  ).�"".�q�� �0���K)��#;���z� *.��",��� "0�


 �
�K)��#;���z� ����������������������������������in��",q�� "4�
J!�
�K)��#���z� ����������������������������������

In [28]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

# Turn binary files into VCF
/home/jupyter/tools/plink \
--bfile pheno_PP2A_EAS \
--recode vcf-fid \
--out pheno_PP2A_EAS

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to pheno_PP2A_EAS.log.
Options in effect:
  --bfile pheno_PP2A_EAS
  --out pheno_PP2A_EAS
  --recode vcf-fid

14998 MB RAM detected; reserving 7499 MB for main workspace.
150 variants loaded from .bim file.
137 people (85 males, 52 females) loaded from .fam.
137 phenotype values loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 137 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%34%35%36%37%38%39%40%41%42%43%44%45%46%47%48%49%50%51%52%53%54%55%56%57%58%59%60%61%62%63%64%65%6

## Annotate PP2A variants

In [29]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR
export PATH=$PATH:/home/jupyter/tools/rvtests/third/tabix-0.2.6/

### Bgzip and Tabix
bgzip pheno_PP2A_EAS.vcf

tabix -f -p vcf pheno_PP2A_EAS.vcf.gz
tabix -f -p vcf pheno_PP2A_EAS.vcf.gz

[bgzip] can't create pheno_PP2A_EAS.vcf.gz: File exists


In [30]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

### Annotate variants using ANNOVAR: https://annovar.openbioinformatics.org/en/latest/ 
perl /home/jupyter/tools/annovar/table_annovar.pl pheno_PP2A_EAS.vcf.gz /home/jupyter/tools/annovar/humandb/ -buildver hg38 \
-out pheno_PP2A_EAS.annovar \
-remove -protocol refGene,clinvar_20140902 \
-operation g,f \
--nopolish \
-nastring . \
-vcfinput


NOTICE: Running with system command <convert2annovar.pl  -includeinfo -allsample -withfreq -format vcf4 pheno_PP2A_EAS.vcf.gz > pheno_PP2A_EAS.annovar.avinput>
NOTICE: Finished reading 157 lines from VCF file
NOTICE: A total of 150 locus in VCF file passed QC threshold, representing 136 SNPs (103 transitions and 33 transversions) and 14 indels/substitutions
NOTICE: Finished writing allele frequencies based on 18632 SNP genotypes (14111 transitions and 4521 transversions) and 1918 indels/substitutions for 137 samples

NOTICE: Running with system command </home/jupyter/tools/annovar/table_annovar.pl pheno_PP2A_EAS.annovar.avinput /home/jupyter/tools/annovar/humandb/ -buildver hg38 -outfile pheno_PP2A_EAS.annovar -remove -protocol refGene,clinvar_20140902 -operation g,f --nopolish -nastring . -otherinfo>
-----------------------------------------------------------------
NOTICE: Processing operation=g protocol=refGene

NOTICE: Running with system command <annotate_variation.pl -geneanno -b

## Extract coding and non-syn variants

In [31]:
# Visualize multianno file
PP2A_EAS = pd.read_csv(f'{WORK_DIR}/pheno_PP2A_EAS.annovar.hg38_multianno.txt', sep = '\t')
PP2A_EAS.head()

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,Otherinfo140,Otherinfo141,Otherinfo142,Otherinfo143,Otherinfo144,Otherinfo145,Otherinfo146,Otherinfo147,Otherinfo148,Otherinfo149
0,9,129110977,129110977,A,C,UTR5,PTPA,NM_021131:c.-624A>C,.,.,...,0/1,0/1,0/1,0/1,0/0,0/0,0/0,0/0,0/1,0/0
1,9,129111323,129111323,C,G,UTR5,PTPA,NM_178001:c.-278C>G;NM_001271832:c.-278C>G,.,.,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
2,9,129111962,129111962,T,C,intronic,PTPA,.,.,.,...,0/0,0/0,0/0,0/1,0/0,0/0,0/1,0/0,0/0,0/0
3,9,129112273,129112273,T,A,intronic,PTPA,.,.,.,...,0/1,0/1,0/1,1/1,./.,0/0,0/1,0/0,0/1,0/0
4,9,129112362,129112362,G,A,intronic,PTPA,.,.,.,...,0/1,0/1,0/1,0/1,0/1,0/0,0/0,0/0,0/1,0/0


In [32]:
PP2A_EAS.count()

Chr             150
Start           150
End             150
Ref             150
Alt             150
               ... 
Otherinfo145    150
Otherinfo146    150
Otherinfo147    150
Otherinfo148    150
Otherinfo149    150
Length: 160, dtype: int64

In [33]:
# Filter exonic variants
coding_EAS = PP2A_EAS[PP2A_EAS['Func.refGene'] == 'exonic']
coding_EAS.count()
coding_EAS.head(5)

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,Otherinfo140,Otherinfo141,Otherinfo142,Otherinfo143,Otherinfo144,Otherinfo145,Otherinfo146,Otherinfo147,Otherinfo148,Otherinfo149
142,9,129147457,129147457,C,T,exonic,PTPA,.,nonsynonymous SNV,"PTPA:NM_001271832:exon9:c.C878T:p.S293L,PTPA:N...",...,0/0,0/1,0/0,0/1,0/0,0/0,0/0,0/0,0/1,0/0


In [34]:
# Filter exonic and non-syn vars
coding_nonsynonymous_EAS = PP2A_EAS[(PP2A_EAS['Func.refGene'] == 'exonic') & (PP2A_EAS['ExonicFunc.refGene'] == 'nonsynonymous SNV')]
coding_nonsynonymous_EAS.count(1)

142    160
dtype: int64

In [35]:
coding_nonsynonymous_EAS.head(1)

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,Otherinfo140,Otherinfo141,Otherinfo142,Otherinfo143,Otherinfo144,Otherinfo145,Otherinfo146,Otherinfo147,Otherinfo148,Otherinfo149
142,9,129147457,129147457,C,T,exonic,PTPA,.,nonsynonymous SNV,"PTPA:NM_001271832:exon9:c.C878T:p.S293L,PTPA:N...",...,0/0,0/1,0/0,0/1,0/0,0/0,0/0,0/0,0/1,0/0


In [36]:
print(coding_nonsynonymous_EAS)

     Chr      Start        End Ref Alt Func.refGene Gene.refGene  \
142    9  129147457  129147457   C   T       exonic         PTPA   

    GeneDetail.refGene ExonicFunc.refGene  \
142                  .  nonsynonymous SNV   

                                      AAChange.refGene  ... Otherinfo140  \
142  PTPA:NM_001271832:exon9:c.C878T:p.S293L,PTPA:N...  ...          0/0   

     Otherinfo141 Otherinfo142 Otherinfo143  Otherinfo144  Otherinfo145  \
142           0/1          0/0          0/1           0/0           0/0   

    Otherinfo146 Otherinfo147 Otherinfo148 Otherinfo149  
142          0/0          0/0          0/1          0/0  

[1 rows x 160 columns]


In [37]:
coding_nonsynonymous_EAS.to_csv(f'{WORK_DIR}/coding_nonsynonymous.txt', sep = '\t', index = False)
coding_nonsynonymous_EAS.head(9)

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,Otherinfo140,Otherinfo141,Otherinfo142,Otherinfo143,Otherinfo144,Otherinfo145,Otherinfo146,Otherinfo147,Otherinfo148,Otherinfo149
142,9,129147457,129147457,C,T,exonic,PTPA,.,nonsynonymous SNV,"PTPA:NM_001271832:exon9:c.C878T:p.S293L,PTPA:N...",...,0/0,0/1,0/0,0/1,0/0,0/0,0/0,0/0,0/1,0/0


In [38]:
# Filter intronic variants
intronic_EAS = PP2A_EAS[PP2A_EAS['Func.refGene'] == 'intronic']
intronic_EAS.count()

Chr             140
Start           140
End             140
Ref             140
Alt             140
               ... 
Otherinfo145    140
Otherinfo146    140
Otherinfo147    140
Otherinfo148    140
Otherinfo149    140
Length: 160, dtype: int64

In [39]:
# Filter UTR3 variants
UTR3_EAS = PP2A_EAS[PP2A_EAS['Func.refGene'] == ('UTR3')]
UTR3_EAS.count()

Chr             7
Start           7
End             7
Ref             7
Alt             7
               ..
Otherinfo145    7
Otherinfo146    7
Otherinfo147    7
Otherinfo148    7
Otherinfo149    7
Length: 160, dtype: int64

In [40]:
# Filter UTR5 variants
UTR5_EAS = PP2A_EAS[PP2A_EAS['Func.refGene'] == ('UTR5')]
UTR5_EAS.count()

Chr             2
Start           2
End             2
Ref             2
Alt             2
               ..
Otherinfo145    2
Otherinfo146    2
Otherinfo147    2
Otherinfo148    2
Otherinfo149    2
Length: 160, dtype: int64

In [41]:
# Calculate freq - cases vs controls

In [42]:
reduced_coding_nonsynonymous_EAS = coding_nonsynonymous_EAS[["Chr", "Start", "End", "Gene.refGene"]]
reduced_coding_nonsynonymous_EAS.to_csv(f'{WORK_DIR}/reduced_coding_nonsynonymous_EAS.txt', sep = '\t', index = False, header= False)

In [43]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

head reduced_coding_nonsynonymous_EAS.txt

9	129147457	129147457	PTPA


In [44]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

/home/jupyter/tools/plink --bfile pheno_PP2A_EAS --extract range reduced_coding_nonsynonymous_EAS.txt --assoc --out coding_nonsynonymous_pheno_PP2A_EAS --allow-no-sex --ci 0.95

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to coding_nonsynonymous_pheno_PP2A_EAS.log.
Options in effect:
  --allow-no-sex
  --assoc
  --bfile pheno_PP2A_EAS
  --ci 0.95
  --extract range reduced_coding_nonsynonymous_EAS.txt
  --out coding_nonsynonymous_pheno_PP2A_EAS

14998 MB RAM detected; reserving 7499 MB for main workspace.
150 variants loaded from .bim file.
137 people (85 males, 52 females) loaded from .fam.
137 phenotype values loaded from .fam.
--extract range: 149 variants excluded.
--extract range: 1 variant remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 137 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%

In [45]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

/home/jupyter/tools/plink --bfile pheno_PP2A_EAS --extract range reduced_coding_nonsynonymous_EAS.txt --make-bed --max-maf 0.05 --out coding_nonsynonymous_EAS_rare_0.05 --allow-no-sex

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to coding_nonsynonymous_EAS_rare_0.05.log.
Options in effect:
  --allow-no-sex
  --bfile pheno_PP2A_EAS
  --extract range reduced_coding_nonsynonymous_EAS.txt
  --make-bed
  --max-maf 0.05
  --out coding_nonsynonymous_EAS_rare_0.05

14998 MB RAM detected; reserving 7499 MB for main workspace.
150 variants loaded from .bim file.
137 people (85 males, 52 females) loaded from .fam.
137 phenotype values loaded from .fam.
--extract range: 149 variants excluded.
--extract range: 1 variant remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 137 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%

Error: All variants removed due to minor allele threshold(s)
(--maf/--max-maf/--mac/--max-mac).


CalledProcessError: Command 'b"\nWORK_DIR='/home/jupyter/PP2A_GP2/'\ncd $WORK_DIR\n\n/home/jupyter/tools/plink --bfile pheno_PP2A_EAS --extract range reduced_coding_nonsynonymous_EAS.txt --make-bed --max-maf 0.05 --out coding_nonsynonymous_EAS_rare_0.05 --allow-no-sex\n"' returned non-zero exit status 13.

In [47]:
PP2A_EAS_freq = pd.read_csv(f'{WORK_DIR}/coding_nonsynonymous_pheno_PP2A_EAS.assoc', delim_whitespace=True)
PP2A_EAS_freq.head(10)

,CHR,SNP,BP,A1,F_A,F_U,A2,CHISQ,P,OR,SE,L95,U95
0,9,chr9:129147457:C:T,129147457,T,0.1307,0.1333,C,0.002757,0.9581,0.9771,0.4407,0.4119,2.318


In [48]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

/home/jupyter/tools/plink --bfile pheno_PP2A_EAS --extract range reduced_coding_nonsynonymous_EAS.txt --assoc --out coding_nonsynonymous_pheno_PP2A_EAS --allow-no-sex --adjust

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to coding_nonsynonymous_pheno_PP2A_EAS.log.
Options in effect:
  --adjust
  --allow-no-sex
  --assoc
  --bfile pheno_PP2A_EAS
  --extract range reduced_coding_nonsynonymous_EAS.txt
  --out coding_nonsynonymous_pheno_PP2A_EAS

14998 MB RAM detected; reserving 7499 MB for main workspace.
150 variants loaded from .bim file.
137 people (85 males, 52 females) loaded from .fam.
137 phenotype values loaded from .fam.
--extract range: 149 variants excluded.
--extract range: 1 variant remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 137 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%

In [49]:
PP2A_EAS_freq = pd.read_csv(f'{WORK_DIR}/coding_nonsynonymous_pheno_PP2A_EAS.assoc.adjusted', delim_whitespace=True)
PP2A_EAS_freq.head(10)

,CHR,SNP,UNADJ,GC,BONF,HOLM,SIDAK_SS,SIDAK_SD,FDR_BH,FDR_BY
0,9,chr9:129147457:C:T,0.9581,0.9581,0.9581,0.9581,0.9581,0.9581,0.9581,0.9581


## Calculate freq of HMZ in cases versus controls

In [50]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

/home/jupyter/tools/plink --bfile pheno_PP2A_EAS --extract range reduced_coding_nonsynonymous_EAS.txt --recode A --out coding_nonsynonymous_pheno_PP2A_EAS

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to coding_nonsynonymous_pheno_PP2A_EAS.log.
Options in effect:
  --bfile pheno_PP2A_EAS
  --extract range reduced_coding_nonsynonymous_EAS.txt
  --out coding_nonsynonymous_pheno_PP2A_EAS
  --recode A

14998 MB RAM detected; reserving 7499 MB for main workspace.
150 variants loaded from .bim file.
137 people (85 males, 52 females) loaded from .fam.
137 phenotype values loaded from .fam.
--extract range: 149 variants excluded.
--extract range: 1 variant remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 137 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%34%35%36%37%

In [51]:
PP2A_EAS_freq = pd.read_csv(f'{WORK_DIR}/coding_nonsynonymous_pheno_PP2A_EAS.assoc', delim_whitespace=True)
PP2A_EAS_freq.head()

,CHR,SNP,BP,A1,F_A,F_U,A2,CHISQ,P,OR
0,9,chr9:129147457:C:T,129147457,T,0.1307,0.1333,C,0.002757,0.9581,0.9771


In [52]:
PP2A_EAS_recode = pd.read_csv(f'{WORK_DIR}/coding_nonsynonymous_pheno_PP2A_EAS.raw', delim_whitespace=True)
PP2A_EAS_recode.head()

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129147457:C:T_T
0,0,APGS_000051_s1,0,0,1,2,NaN
1,0,APGS_000113_s1,0,0,1,2,1.0
2,0,APGS_000135_s1,0,0,1,2,0.0
3,0,APGS_000186_s1,0,0,1,2,0.0
4,0,APGS_000201_s1,0,0,1,2,1.0


In [53]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

/home/jupyter/tools/plink --bfile  pheno_PP2A_EAS --extract range reduced_coding_nonsynonymous_EAS.txt --recode A --out coding_nonsynonymous_pheno_PP2A_EAS

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to coding_nonsynonymous_pheno_PP2A_EAS.log.
Options in effect:
  --bfile pheno_PP2A_EAS
  --extract range reduced_coding_nonsynonymous_EAS.txt
  --out coding_nonsynonymous_pheno_PP2A_EAS
  --recode A

14998 MB RAM detected; reserving 7499 MB for main workspace.
150 variants loaded from .bim file.
137 people (85 males, 52 females) loaded from .fam.
137 phenotype values loaded from .fam.
--extract range: 149 variants excluded.
--extract range: 1 variant remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 137 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%34%35%36%37%

In [54]:
# Explore HMZs for chr9:129147457:C:T_T in cases
PP2A_hom_casos_EAS = PP2A_EAS_recode[(PP2A_EAS_recode['chr9:129147457:C:T_T'] == 2) & (PP2A_EAS_recode['PHENOTYPE'] == 2)]
PP2A_hom_casos_EAS.head()

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129147457:C:T_T
13,0,APGS_000704_s1,0,0,1,2,2.0


In [55]:
PP2A_hom_casos_EAS.shape

(1, 7)

In [56]:
# Explore HMZs for chr9:129147457:C:T_T in controls
PP2A_hom_controles_EAS = PP2A_EAS_recode[(PP2A_EAS_recode['chr9:129147457:C:T_T'] == 2) & (PP2A_EAS_recode['PHENOTYPE'] == 1)]
PP2A_hom_controles_EAS.head()

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129147457:C:T_T


In [57]:
PP2A_hom_controles_EAS.shape

(0, 7)

In [58]:
# Explore HETs for chr9:129147457:C:T_T in cases
PP2A_het_casos_129147457_EAS = PP2A_EAS_recode[(PP2A_EAS_recode['chr9:129147457:C:T_T'] == 1) & (PP2A_EAS_recode['PHENOTYPE'] == 2)]
PP2A_het_casos_129147457_EAS

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129147457:C:T_T
1,0,APGS_000113_s1,0,0,1,2,1.0
4,0,APGS_000201_s1,0,0,1,2,1.0
5,0,APGS_000204_s1,0,0,2,2,1.0
10,0,APGS_000531_s1,0,0,2,2,1.0
16,0,APGS_000741_s1,0,0,2,2,1.0
28,0,APGS_001100_s1,0,0,2,2,1.0
32,0,APGS_001167_s1,0,0,1,2,1.0
37,0,APGS_001416_s1,0,0,2,2,1.0
47,0,BCM_000661_s1,0,0,1,2,1.0
52,0,CORIELL_000731_s1,0,0,1,2,1.0


In [59]:
# Explore HETs for chr9:129147457:C:T_T in controls
PP2A_het_controles_129147457_EAS = PP2A_EAS_recode[(PP2A_EAS_recode['chr9:129147457:C:T_T'] == 1) & (PP2A_EAS_recode['PHENOTYPE'] == 1)]
PP2A_het_controles_129147457_EAS

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129147457:C:T_T
42,0,BCM_000172_s1,0,0,2,1,1.0
92,0,CORIELL_006204_s1,0,0,2,1,1.0
95,0,CORIELL_006495_s1,0,0,1,1,1.0
96,0,CORIELL_006718_s1,0,0,2,1,1.0
100,0,CORIELL_006995_s1,0,0,1,1,1.0
102,0,CORIELL_007064_s1,0,0,2,1,1.0
104,0,CORIELL_007232_s1,0,0,2,1,1.0
123,0,PAGE_002732_s1,0,0,1,1,1.0


In [61]:
!pip install rpy2
%load_ext rpy2.ipython

In [62]:
import rpy2
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [63]:
%%R
setwd('/home/jupyter/PP2A_GP2/')
library(data.table)
covs <- fread("master_key_release3_final.csv", header =T)
covs

                  GP2ID         GP2sampleID manifest_id phenotype pheno_for_qc
    1: SYNAPS-KZ_000001 SYNAPS-KZ_000001_s1          m1   Control            1
    2: SYNAPS-KZ_000002 SYNAPS-KZ_000002_s1          m1   Control            1
    3: SYNAPS-KZ_000003 SYNAPS-KZ_000003_s1          m1   Control            1
    4: SYNAPS-KZ_000004 SYNAPS-KZ_000004_s1          m1   Control            1
    5: SYNAPS-KZ_000005 SYNAPS-KZ_000005_s1          m1   Control            1
   ---                                                                        
16853:      PPMI_000789      PPMI_000789_s1          m2     Other           -9
16854:      PPMI_000781      PPMI_000781_s1          m2     Other           -9
16855:      PPMI_000818      PPMI_000818_s1          m2     Other           -9
16856:      PPMI_000841      PPMI_000841_s1          m2     Other           -9
16857:      PPMI_000808      PPMI_000808_s1          m2     Other           -9
                                  other_pheno sex_fo

data.table 1.14.6 using 2 threads (see ?getDTthreads).  Latest news: r-datatable.com
In addition: Warning message:
In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
  libraries ‘/home/jupyter/packages’, ‘/usr/lib/R/site-library’ contain no packages


In [64]:
%%R
colnames(covs)[2]  <- "IID"  
covs

                  GP2ID                 IID manifest_id phenotype pheno_for_qc
    1: SYNAPS-KZ_000001 SYNAPS-KZ_000001_s1          m1   Control            1
    2: SYNAPS-KZ_000002 SYNAPS-KZ_000002_s1          m1   Control            1
    3: SYNAPS-KZ_000003 SYNAPS-KZ_000003_s1          m1   Control            1
    4: SYNAPS-KZ_000004 SYNAPS-KZ_000004_s1          m1   Control            1
    5: SYNAPS-KZ_000005 SYNAPS-KZ_000005_s1          m1   Control            1
   ---                                                                        
16853:      PPMI_000789      PPMI_000789_s1          m2     Other           -9
16854:      PPMI_000781      PPMI_000781_s1          m2     Other           -9
16855:      PPMI_000818      PPMI_000818_s1          m2     Other           -9
16856:      PPMI_000841      PPMI_000841_s1          m2     Other           -9
16857:      PPMI_000808      PPMI_000808_s1          m2     Other           -9
                                  other_pheno sex_fo

In [65]:
PP2A_het_casos_129147457_EAS.to_csv('PP2A_het_casos_129147457_EAS.csv')

In [66]:
temp = pd.read_csv('PP2A_het_casos_129147457_EAS.csv')
temp

,Unnamed: 0,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129147457:C:T_T
0,1,0,APGS_000113_s1,0,0,1,2,1.0
1,4,0,APGS_000201_s1,0,0,1,2,1.0
2,5,0,APGS_000204_s1,0,0,2,2,1.0
3,10,0,APGS_000531_s1,0,0,2,2,1.0
4,16,0,APGS_000741_s1,0,0,2,2,1.0
5,28,0,APGS_001100_s1,0,0,2,2,1.0
6,32,0,APGS_001167_s1,0,0,1,2,1.0
7,37,0,APGS_001416_s1,0,0,2,2,1.0
8,47,0,BCM_000661_s1,0,0,1,2,1.0
9,52,0,CORIELL_000731_s1,0,0,1,2,1.0


In [67]:
PP2A_het_controles_129147457_EAS.to_csv('PP2A_het_controles_129147457_EAS.csv')

In [68]:
temp = pd.read_csv('PP2A_het_controles_129147457_EAS.csv')
temp

,Unnamed: 0,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129147457:C:T_T
0,42,0,BCM_000172_s1,0,0,2,1,1.0
1,92,0,CORIELL_006204_s1,0,0,2,1,1.0
2,95,0,CORIELL_006495_s1,0,0,1,1,1.0
3,96,0,CORIELL_006718_s1,0,0,2,1,1.0
4,100,0,CORIELL_006995_s1,0,0,1,1,1.0
5,102,0,CORIELL_007064_s1,0,0,2,1,1.0
6,104,0,CORIELL_007232_s1,0,0,2,1,1.0
7,123,0,PAGE_002732_s1,0,0,1,1,1.0


In [69]:
%%R
setwd('/home/jupyter/PP2A_GP2/')
library(data.table)
PP2A_het_controles_129147457_EAS <- fread("PP2A_het_controles_129147457_EAS.csv", header =T)
PP2A_het_controles_129147457_EAS
temp <- merge(covs, PP2A_het_controles_129147457_EAS, by="IID")
head(temp)
write.table(temp, file = "PP2A_het_controles_129147457_EAS_extracted.txt", quote = F, row.names = F, sep = "\t")
PP2A_het_controles_129147457_EAS_extracted <- fread("PP2A_het_controles_129147457_EAS_extracted.txt", header =T)
PP2A_het_controles_129147457_EAS_extracted

                 IID          GP2ID manifest_id phenotype pheno_for_qc
1:     BCM_000172_s1     BCM_000172          m1   Control            1
2: CORIELL_006204_s1 CORIELL_006204          m6   Control            1
3: CORIELL_006495_s1 CORIELL_006495          m6   Control            1
4: CORIELL_006718_s1 CORIELL_006718          m6   Control            1
5: CORIELL_006995_s1 CORIELL_006995          m6   Control            1
6: CORIELL_007064_s1 CORIELL_007064          m6   Control            1
7: CORIELL_007232_s1 CORIELL_007232          m6   Control            1
8:    PAGE_002732_s1    PAGE_002732          m1   Control            1
   other_pheno sex_for_qc age age_of_onset age_at_diagnosis age_at_death
1:     Control          2  NA           NA               NA           NA
2:     Control          2  70           NA               NA           NA
3:     Control          1  62           NA               NA           NA
4:     Control          2  71           NA               NA          

In [70]:
%%R
setwd('/home/jupyter/PP2A_GP2/')
library(data.table)
PP2A_het_casos_129147457_EAS <- fread("PP2A_het_casos_129147457_EAS.csv", header =T)
PP2A_het_casos_129147457_EAS
temp <- merge(covs, PP2A_het_casos_129147457_EAS, by="IID")
head(temp)
write.table(temp, file = "PP2A_het_casos_129147457_EAS_extracted.txt", quote = F, row.names = F, sep = "\t")
PP2A_het_casos_129147457_EAS_extracted <- fread("PP2A_het_casos_129147457_EAS_extracted.txt", header =T)
PP2A_het_casos_129147457_EAS_extracted

                  IID          GP2ID manifest_id phenotype pheno_for_qc
 1:    APGS_000113_s1    APGS_000113          m1        PD            2
 2:    APGS_000201_s1    APGS_000201          m1        PD            2
 3:    APGS_000204_s1    APGS_000204          m1        PD            2
 4:    APGS_000531_s1    APGS_000531          m1        PD            2
 5:    APGS_000741_s1    APGS_000741          m1        PD            2
 6:    APGS_001100_s1    APGS_001100          m1        PD            2
 7:    APGS_001167_s1    APGS_001167          m1        PD            2
 8:    APGS_001416_s1    APGS_001416          m1        PD            2
 9:     BCM_000661_s1     BCM_000661          m1        PD            2
10: CORIELL_000731_s1 CORIELL_000731          m3        PD            2
11: CORIELL_001792_s1 CORIELL_001792          m3        PD            2
12: CORIELL_003150_s1 CORIELL_003150          m3        PD            2
13: CORIELL_003161_s1 CORIELL_003161          m3        PD      

## Save out results..!

In [71]:
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r {WORK_DIR} {WORKSPACE_BUCKET}')

Executing: gsutil -mu terra-02e7cd23 cp -r /home/jupyter/PP2A_GP2/ gs://fc-a2382a6f-59cf-4cf3-b70e-8cd83806d5e0


Copying file:///home/jupyter/PP2A_GP2/reduced_coding_nonsynonymous.txt [Content-Type=text/plain]...
Copying file:///home/jupyter/PP2A_GP2/pheno_PP2A.annovar.hg38_multianno.txt [Content-Type=text/plain]...
Copying file:///home/jupyter/PP2A_GP2/PP2A_het_controles_129147457_EUR_extracted.txt [Content-Type=text/plain]...
Copying file:///home/jupyter/PP2A_GP2/pheno_PP2A_AMR.vcf.gz [Content-Type=text/vcard]...
Copying file:///home/jupyter/PP2A_GP2/coding_nonsynonymous_AJ_rare_0.05.log [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/PP2A_GP2/pheno_PP2A_CAS.log [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/PP2A_GP2/PP2A_het_casos_129147457_CAS.csv [Content-Type=text/csv]...
Copying file:///home/jupyter/PP2A_GP2/pheno_PP2A_AFR.fam [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/PP2A_GP2/pheno_PP2A.vcf [Content-Type=text/vcard]...
Copying file:///home/jupyter/PP2A_GP2/coding_nonsynonymous_AAC_rare_0.05.bed [Content-Type=ap